# * ETL-Actual MKS : Broadband
    (P, G, H, HH, Z) level

## Parameter

In [2]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import oracledb
import re
import FN_Actual_MKS_Broadband as fn

config = configparser.ConfigParser()
config.read('../../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
next_week_dt = curr_dt + dt.timedelta(days=7)
str_curr_dt = curr_dt.strftime('%Y%m%d')
str_next_week_dt = next_week_dt.strftime('%Y%m%d')
yyyymm_curr = curr_dt.strftime('%Y%m')
yyyymm_next_week = next_week_dt.strftime('%Y%m')

### Lasted Source Summary

In [3]:
# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()


try:
    # Get : Actual Summary from "FCT_BB_SHARE_SUBS_CCAATT"
    src_cur.execute("""
        SELECT TM_KEY_MTH
            , SUM(SUBS) TOTAL
            , SUM(CASE WHEN ISP = 'TOL' THEN SUBS END) TOL
            , SUM(CASE WHEN ISP = '3BB' THEN SUBS END) "3BB"
            , SUM(CASE WHEN ISP = 'AIS' THEN SUBS END) AIS
            , SUM(CASE WHEN ISP IN ('CAT', 'TOT') THEN SUBS END) NT
        FROM CORPNSBOX.FCT_BB_SHARE_SUBS_CCAATT A
        WHERE TM_KEY_MTH >= 202401
        --WHERE TM_KEY_MTH = (SELECT MAX(TM_KEY_MTH) FROM CORPNSBOX.FCT_BB_SHARE_SUBS_CCAATT NOLOCK)
        AND EXISTS (SELECT 1 FROM CDSAPPO.DIM_MOOC_AREA O
			        WHERE O.REMARK <> 'Dummy'
			        AND O.CCAATT = A.CCAATT)
        GROUP BY TM_KEY_MTH
        ORDER BY 1
    """)
    rows = src_cur.fetchall()
    print(f'\nCurrent Source Summary...')
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    tmp_src_df = chk_src_df.copy()
    mod_col_list = tmp_src_df.iloc[:, 1:6].columns.tolist()
    for col in mod_col_list:
        tmp_src_df[col] = tmp_src_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{tmp_src_df}')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Current Source Summary...

   TM_KEY_MTH      TOTAL        TOL        3BB        AIS         NT
0      202401  8,392,163  3,062,361  2,206,741  1,703,305  1,419,756
1      202402  8,399,105  3,065,282  2,197,017  1,716,028  1,420,779
2      202403  8,414,203  3,072,991  2,187,215  1,732,168  1,421,829
3      202404  8,422,602  3,073,168  2,177,393  1,749,183  1,422,858
4      202405  8,439,795  3,082,458  2,167,565  1,765,884  1,423,888


### Lasted Fact Summary

In [4]:
# Connect : AKPIPRD
tgt_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
tgt_conn = oracledb.connect(tgt_dsn)
tgt_cur = tgt_conn.cursor()


try:
    # Get : Actual Summary from "FCT_BROADBAND_MKS"
    tgt_cur.execute("""
        SELECT TM_KEY_MTH
            , SUM(CASE WHEN METRIC_CD IN ('VIN00025', 'VIN00026', 'VIN00027', 'VIN00028') THEN METRIC_VALUE END) TOTAL
            , SUM(CASE WHEN METRIC_CD = 'VIN00025' THEN METRIC_VALUE END) TOL
            , SUM(CASE WHEN METRIC_CD = 'VIN00026' THEN METRIC_VALUE END) "3BB"
            , SUM(CASE WHEN METRIC_CD = 'VIN00027' THEN METRIC_VALUE END) AIS
            , SUM(CASE WHEN METRIC_CD = 'VIN00028' THEN METRIC_VALUE END) NT
            --, MAX(LOAD_DATE) LOAD_DATE
        FROM AUTOKPI.FCT_BROADBAND_MKS NOLOCK
        WHERE TM_KEY_MTH >= 202401 --AND TM_KEY_MTH <= 202404
        AND TM_KEY_DAY LIKE '%01'
        AND AREA_TYPE = 'P'
        AND REMARK IS NULL
        GROUP BY TM_KEY_MTH
        ORDER BY 1
    """)
    rows = tgt_cur.fetchall()
    print(f'\nCurrent Fact Summary...')
    chk_tgt_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in tgt_cur.description])

    # Display
    tmp_tgt_df = chk_tgt_df.copy()
    mod_col_list = tmp_tgt_df.iloc[:, 1:6].columns.tolist()
    for col in mod_col_list:
        tmp_tgt_df[col] = tmp_tgt_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{tmp_tgt_df}')

    # Get : MAX(TM_KEY_MTH)
    tgt_cur.execute("SELECT MAX(TM_KEY_MTH) FROM AUTOKPI.FCT_BROADBAND_MKS NOLOCK")
    max_mth_fct = tgt_cur.fetchone()
    print(f'\nv_max_mth_fct: {max_mth_fct[0]}')
    
    tgt_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    tgt_conn.close()


Current Fact Summary...

   TM_KEY_MTH      TOTAL        TOL        3BB        AIS         NT
0      202401  8,392,163  3,062,361  2,206,741  1,703,305  1,419,756
1      202402  8,399,105  3,065,282  2,197,017  1,716,028  1,420,779
2      202403  8,414,203  3,072,991  2,187,215  1,732,168  1,421,829
3      202404  8,422,602  3,073,168  2,177,393  1,749,183  1,422,858
4      202405  8,439,795  3,082,458  2,167,565  1,765,884  1,423,888

v_max_mth_fct: 202405


### Check Diff Summary

In [5]:
chk_diff_df = chk_src_df.iloc[-1].compare(chk_tgt_df.iloc[-1])
chk_diff_df['diff'] = chk_diff_df['self'] - chk_diff_df['other']
mod_col_list = chk_diff_df.columns.tolist()
for col in mod_col_list:
    chk_diff_df[col] = chk_diff_df[col].apply(lambda x: format(x, ',.0f'))

chk_diff_df

,self,other,diff


### Input Parameter

In [6]:
# v_update_flag = 'Y' # Test

v_update_flag = 'Y' if chk_diff_df.size > 0 else 'N'
v_mth_end_src = chk_src_df['TM_KEY_MTH'].max().astype(float)
v_mth_end_fct = chk_tgt_df['TM_KEY_MTH'].max().astype(float)
v_max_mth_fct = max_mth_fct[0]
v_mth_end_fct_year = pd.to_datetime(v_mth_end_fct, format='%Y%m').year
v_mth_end_fct_month = pd.to_datetime(v_mth_end_fct, format='%Y%m').month
v_prev_mth_fct = v_mth_end_fct-1 if v_mth_end_fct_month != 1 else float(str(v_mth_end_fct_year-1)+str(12))

print(f'yyyymm_next_week: {yyyymm_next_week}')
print(f'v_update_flag: {v_update_flag}')
print(f'v_mth_end_src: {v_mth_end_src}')
print(f'v_mth_end_fct: {v_mth_end_fct}')
print(f'v_max_mth_fct: {v_max_mth_fct}')
print(f'v_prev_mth_fct: {v_prev_mth_fct}')

yyyymm_next_week: 202407
v_update_flag: N
v_mth_end_src: 202405.0
v_mth_end_fct: 202405.0
v_max_mth_fct: 202405
v_prev_mth_fct: 202404.0


## ETL Process...

### DB source to DB Target
    Delete -> Insert

    Source : CORPNSBOX.FCT_BB_SHARE_SUBS_CCAATT
             CDSAPPO.DIM_MOOC_AREA
             CDSAPPO.DIM_TIME
    
    Target : AUTOKPI.FCT_BROADBAND_MKS

In [7]:
# Create Param
v_target_schema = 'AUTOKPI'
v_target_table = 'FCT_BROADBAND_MKS'
v_sql_rawdata_fact = 'Raw-FCT_BROADBAND_MKS.sql'
v_sql_mockup_fact = 'Mock-FCT_BROADBAND_MKS.sql'

job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nJob Start... {job_start_datetime}')

# Process flow
if v_update_flag == 'Y':
    if yyyymm_next_week > str(v_mth_end_fct):
        print(f'\n*** Source update & mockup to next month Fact ***')
        fn.src_update_to_fact(v_mth_end_fct, v_target_schema, v_target_table, v_sql_rawdata_fact)
    else:
        print(f'\n*** Source update current month Fact ***')
        fn.src_update_to_fact(v_prev_mth_fct, v_target_schema, v_target_table, v_sql_rawdata_fact)

elif v_update_flag == 'N':
    if yyyymm_next_week > str(v_max_mth_fct):
        print(f'\n*** Last Fact mockup to next month ***')
        fn.mockup_to_fact(v_max_mth_fct, v_target_schema, v_target_table, v_sql_mockup_fact)
    else:
        print(f'\n*** Not update ***')

print(f'\nJob Done !!!')


Job Start... 2024-06-24, 15:32:33

*** Last Fact mockup to next month ***

Param input...

   -> mth_end_fct: 202405
   -> target_schema: AUTOKPI
   -> target_table: FCT_BROADBAND_MKS
   -> sql_mockup_fact: Mock-FCT_BROADBAND_MKS.sql
   -> v_param: {'mth_end_fct': 202405}

AKPIPRD : Connected

Processing...

   -> DELETE : "FCT_BROADBAND_MKS" : Done !

   -> INSERT : "FCT_BROADBAND_MKS" : Done !

AKPIPRD : Disconnected

Job Done !!!


## Check Result : "FCT_BROADBAND_MKS"

In [24]:
''' Create Result DataFrame '''

# Connect : AKPIPRD
tgt_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
tgt_conn = oracledb.connect(tgt_dsn)
tgt_cur = tgt_conn.cursor()
# print(f'\n{AKPIPRD_db} : Connected')


try:
    # Get : Result Data Summary
    tgt_cur.execute("""
        SELECT TM_KEY_MTH, AREA_NO, AREA_TYPE, COALESCE(REMARK, 'Actual') REMARK
            -->> Subs
            , SUM(CASE WHEN METRIC_CD IN ('VIN00025', 'VIN00026', 'VIN00027', 'VIN00028') THEN METRIC_VALUE END) TOTAL
            , SUM(CASE WHEN METRIC_CD = 'VIN00024' THEN METRIC_VALUE END) "AIS & 3BB"
            , SUM(CASE WHEN METRIC_CD = 'VIN00025' THEN METRIC_VALUE END) TOL
            , SUM(CASE WHEN METRIC_CD = 'VIN00026' THEN METRIC_VALUE END) "3BB"
            , SUM(CASE WHEN METRIC_CD = 'VIN00027' THEN METRIC_VALUE END) AIS
            , SUM(CASE WHEN METRIC_CD = 'VIN00028' THEN METRIC_VALUE END) NT
            -->> % MKS
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD IN ('VIN00020', 'VIN00021', 'VIN00022', 'VIN00023') THEN METRIC_VALUE END) "% TOTAL"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00019' THEN METRIC_VALUE END) "% AIS & 3BB"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00020' THEN METRIC_VALUE END) "% TOL"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00021' THEN METRIC_VALUE END) "% 3BB"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00022' THEN METRIC_VALUE END) "% AIS"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00023' THEN METRIC_VALUE END) "% NT"
            -->> Count
            , COUNT(DISTINCT METRIC_CD) CNT_METRIC, COUNT(1) ROW_CNT, MAX(LOAD_DATE) LOAD_DATE
        FROM AUTOKPI.FCT_BROADBAND_MKS NOLOCK
        WHERE TM_KEY_DAY LIKE '%01'
        GROUP BY TM_KEY_MTH, AREA_NO, AREA_TYPE, REMARK
        ORDER BY 1,2
    """)
    rows = tgt_cur.fetchall()
    print(f'\nGet : Fact Summary...')
    chk_result_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in tgt_cur.description])
    print(f'\n   -> chk_result_df : {chk_result_df.shape[0]} rows, {chk_result_df.shape[1]} columns') 
    
    # Display
    tmp_result_df = chk_result_df.copy()
    # tmp_result_df = tmp_result_df.replace(np.nan, None)
    # tmp_result_df.iloc[:, 4:18] = tmp_result_df.iloc[:, 4:18].fillna(0)
    mod_col_list = tmp_result_df.iloc[:, 4:18].columns.tolist()
    for col in mod_col_list:
        tmp_result_df[col] = tmp_result_df[col].apply(lambda x: format(x, ',.2f') if re.search('%', col) else format(x, ',.0f'))
        
    tgt_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    tgt_conn.close()


Get : Fact Summary...

   -> chk_result_df : 49 rows, 19 columns


In [25]:
chk_result_df.tail(3)

,TM_KEY_MTH,AREA_NO,AREA_TYPE,REMARK,TOTAL,AIS & 3BB,TOL,3BB,AIS,NT,% TOTAL,% AIS & 3BB,% TOL,% 3BB,% AIS,% NT,CNT_METRIC,ROW_CNT,LOAD_DATE
46,202407,5,G,Data as of : 202405,8.439795e+06,3.933449e+06,3082458,2.167565e+06,1765884.072,1.423888e+06,NaN,NaN,NaN,NaN,NaN,NaN,10,80,2024-06-24 15:33:06.182638
47,202407,6,H,Data as of : 202405,8.439795e+06,3.933449e+06,3082458,2.167565e+06,1765884.072,1.423888e+06,NaN,NaN,NaN,NaN,NaN,NaN,10,650,2024-06-24 15:33:06.182638
48,202407,7,HH,Data as of : 202405,8.439795e+06,3.933449e+06,3082458,2.167565e+06,1765884.072,1.423888e+06,NaN,NaN,NaN,NaN,NaN,NaN,10,960,2024-06-24 15:33:06.182638


In [26]:
tmp_result_df.tail(3)

,TM_KEY_MTH,AREA_NO,AREA_TYPE,REMARK,TOTAL,AIS & 3BB,TOL,3BB,AIS,NT,% TOTAL,% AIS & 3BB,% TOL,% 3BB,% AIS,% NT,CNT_METRIC,ROW_CNT,LOAD_DATE
46,202407,5,G,Data as of : 202405,"8,439,795","3,933,449","3,082,458","2,167,565","1,765,884","1,423,888",nan,nan,nan,nan,nan,nan,10,80,2024-06-24 15:33:06.182638
47,202407,6,H,Data as of : 202405,"8,439,795","3,933,449","3,082,458","2,167,565","1,765,884","1,423,888",nan,nan,nan,nan,nan,nan,10,650,2024-06-24 15:33:06.182638
48,202407,7,HH,Data as of : 202405,"8,439,795","3,933,449","3,082,458","2,167,565","1,765,884","1,423,888",nan,nan,nan,nan,nan,nan,10,960,2024-06-24 15:33:06.182638


In [27]:
''' Filters '''

max_mth = chk_result_df['TM_KEY_MTH'].max()

rec_df = chk_result_df
# rec_df = tmp_result_df.loc[tmp_result_df['TM_KEY_MTH']==max_mth]

rec_df.tail()

,TM_KEY_MTH,AREA_NO,AREA_TYPE,REMARK,TOTAL,AIS & 3BB,TOL,3BB,AIS,NT,% TOTAL,% AIS & 3BB,% TOL,% 3BB,% AIS,% NT,CNT_METRIC,ROW_CNT,LOAD_DATE
44,202407,3,Z,Data as of : 202405,8.392859e+05,4.267935e+05,273542,2.308766e+05,1.959170e+05,1.389504e+05,NaN,NaN,NaN,NaN,NaN,NaN,10,10,2024-06-24 15:33:06.182638
45,202407,4,Z,Data as of : 202405,2.810569e+06,1.050291e+06,1372201,4.556779e+05,5.946135e+05,3.880767e+05,NaN,NaN,NaN,NaN,NaN,NaN,10,40,2024-06-24 15:33:06.182638
46,202407,5,G,Data as of : 202405,8.439795e+06,3.933449e+06,3082458,2.167565e+06,1.765884e+06,1.423888e+06,NaN,NaN,NaN,NaN,NaN,NaN,10,80,2024-06-24 15:33:06.182638
47,202407,6,H,Data as of : 202405,8.439795e+06,3.933449e+06,3082458,2.167565e+06,1.765884e+06,1.423888e+06,NaN,NaN,NaN,NaN,NaN,NaN,10,650,2024-06-24 15:33:06.182638
48,202407,7,HH,Data as of : 202405,8.439795e+06,3.933449e+06,3082458,2.167565e+06,1.765884e+06,1.423888e+06,NaN,NaN,NaN,NaN,NaN,NaN,10,960,2024-06-24 15:33:06.182638


In [29]:
agg_df = rec_df.groupby(['REMARK', 'TM_KEY_MTH', 'CNT_METRIC']).agg({'AREA_TYPE': 'count', 'ROW_CNT': 'sum', 'LOAD_DATE': 'max'}).reset_index()
agg_df

,REMARK,TM_KEY_MTH,CNT_METRIC,AREA_TYPE,ROW_CNT,LOAD_DATE
0,Actual,202401,10,7,1770,2024-06-23 12:42:54.184543
1,Actual,202402,10,7,1770,2024-06-23 12:42:54.184543
2,Actual,202403,10,7,1770,2024-06-23 12:42:54.184543
3,Actual,202404,10,7,1770,2024-06-23 12:42:54.184543
4,Actual,202405,10,7,1770,2024-06-24 00:14:36.089679
5,Data as of : 202405,202406,10,7,1770,2024-06-24 15:33:06.182638
6,Data as of : 202405,202407,10,7,1770,2024-06-24 15:33:06.182638
